<a href="https://colab.research.google.com/github/dinithmaleesha/MLAI-CW/blob/main/CW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment 2

### Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [2]:
dataset = pd.read_csv('/content/Performance.csv')

In [3]:
dataset.isna().sum()

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [4]:
dataset.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


In [5]:
dataset.shape

(10000, 6)

In [6]:
encoder = LabelEncoder()
dataset["Extracurricular Activities"] =  encoder.fit_transform(dataset["Extracurricular Activities"])

In [7]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Multiple Linear Regression

In [9]:
model_mlr = LinearRegression()
model_mlr.fit(X_train, y_train)

y_pred_mlr = model_mlr.predict(X_test)

r2_mlr = r2_score(y_test, y_pred_mlr)
print("R-squared score:", r2_mlr)

R-squared score: 0.9889832909573145


### SVR Model



In [10]:
svr_regressor = SVR()
svr_regressor.fit(X_train, y_train)

SVR()

In [ ]:
param_distribs = [
    {
        'kernel': ['rbf', 'poly'],
        'C': reciprocal(20, 200000),
        'gamma': expon(scale=1.0),
        'degree': expon(scale=1.0)
    },
    {
        'kernel': ['linear', 'sigmoid'],
        'C': reciprocal(20, 200000)
    }
]

model_svr = SVR()

try:
    time_sta = time.time()
    rnd_search_svr = RandomizedSearchCV(model_svr,
                                        param_distributions=param_distribs,
                                        n_iter=50, cv=5, scoring='r2',
                                        verbose=2, random_state=42)
    rnd_search_svr.fit(X, y)
    time_end = time.time()
    tim = time_end - time_sta

    print("Best Parameters:", rnd_search_svr.best_params_)
    print("Time taken:", tim)
except ValueError as e:
    print("Error:", e)
    print("Try adjusting hyperparameter ranges or other configurations.")


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END C=30704.49388394695, degree=0.2026485043181491, gamma=1.5127241738646855, kernel=rbf; total time=   0.0s
[CV] END C=30704.49388394695, degree=0.2026485043181491, gamma=1.5127241738646855, kernel=rbf; total time=   0.0s
[CV] END C=30704.49388394695, degree=0.2026485043181491, gamma=1.5127241738646855, kernel=rbf; total time=   0.0s
[CV] END C=30704.49388394695, degree=0.2026485043181491, gamma=1.5127241738646855, kernel=rbf; total time=   0.0s
[CV] END C=30704.49388394695, degree=0.2026485043181491, gamma=1.5127241738646855, kernel=rbf; total time=   0.0s
[CV] END C=1214.3978986882596, degree=0.10533264473289401, gamma=0.6147961650819368, kernel=rbf; total time=   0.0s
[CV] END C=1214.3978986882596, degree=0.10533264473289401, gamma=0.6147961650819368, kernel=rbf; total time=   0.0s
[CV] END C=1214.3978986882596, degree=0.10533264473289401, gamma=0.6147961650819368, kernel=rbf; total time=   0.0s
[CV] END C=1214.3978

In [ ]:
svr = SVR(C=5, gamma='auto', kernel= 'linear')
svr.fit(X_train, y_train)

y_pred_svr = svr.predict(X_test)
r2_svr = r2_score(y_test, y_pred_svr)
print("R-squared score:", r2_svr)

R-squared score: 0.9880365970819784


### Decision Tree Regression



In [ ]:
model_dt = DecisionTreeRegressor(random_state=0)
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)

r2 = r2_score(y_test, y_pred_dt)
print("R-squared score:", r2)

R-squared score: 0.9743089636107535


### Random Forest Regression

In [ ]:
param_grid = {'n_estimators': [10, 50, 100, 200]}

model_rf = RandomForestRegressor()

grid_search = GridSearchCV(model_rf, param_grid, cv=5, scoring='r2')

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred_rf = best_model.predict(X_test)

r2 = r2_score(y_test, y_pred_rf)
print("Best n_estimators:", grid_search.best_params_['n_estimators'])
print("R-squared score (Grid Search CV):", r2)

Best n_estimators: 200
R-squared score (Grid Search CV): 0.9842954637925958
